In [39]:
# 导入必要的库
import pandas as pd
import numpy as np

# 读取CSV文件
df = pd.read_csv('charging_station_data_20240710_185957.csv')
df = df.reset_index(drop=True)
# 显示数据的基本信息
print(df.info())

# 检查tariff_amount列的空值数量
print("\n空值数量:")
print(df['tariff_amount'].isnull().sum())

# 定义一个函数来填充tariff_amount的空值
def fill_tariff_amount(group):
    mask = group['tariff_amount'].isnull()
    group.loc[mask, 'tariff_amount'] = group['tariff_amount'].mean()
    return group

# 按max_chargerate, plug_type, connector_type分组，并应用填充函数
df = df.groupby(['max_chargerate', 'plug_type', 'connector_type'], group_keys=False).apply(fill_tariff_amount)

# 再次检查tariff_amount列的空值数量
print("\n填充后的空值数量:")
print(df['tariff_amount'].isnull().sum())

# 显示填充后的数据样本
print("\n填充后的数据样本:")
print(df[['max_chargerate', 'plug_type', 'connector_type', 'tariff_amount']].sample(10))

# 检查填充是否正确
print("\n每组的平均tariff_amount:")
print(df.groupby(['max_chargerate', 'plug_type', 'connector_type'])['tariff_amount'].mean())

C:\Users\85407\AppData\Local\Temp\ipykernel_19288\3427081545.py:6: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('charging_station_data_20240710_185957.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 654118 entries, 0 to 654117
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   city_id               654118 non-null  int64  
 1   station_name          654118 non-null  object 
 2   connector_id          654118 non-null  int64  
 3   coordinates_x         654118 non-null  float64
 4   coordinates_y         654118 non-null  float64
 5   postcode              654118 non-null  object 
 6   tariff_amount         632710 non-null  float64
 7   tariff_connectionfee  241040 non-null  float64
 8   max_chargerate        654118 non-null  object 
 9   plug_type             654118 non-null  object 
 10  connector_type        654118 non-null  object 
 11  datetime              654118 non-null  object 
 12  weather               347450 non-null  object 
 13  is_available          654118 non-null  int64  
dtypes: float64(4), int64(3), object(7)
memory usage: 69.

In [41]:
# 检查 tariff_connectionfee 列的空值数量
print("\ntariff_connectionfee 列的空值数量(填充前):")
print(df['tariff_connectionfee'].isnull().sum())

# 将 tariff_connectionfee 列的空值填充为 0
df['tariff_connectionfee'] = df['tariff_connectionfee'].fillna(0)

# 再次检查 tariff_connectionfee 列的空值数量,确保填充成功
print("\ntariff_connectionfee 列的空值数量(填充后):")
print(df['tariff_connectionfee'].isnull().sum())

# 显示填充后的数据样本
print("\n填充后的数据样本:")
print(df[['station_name', 'tariff_amount', 'tariff_connectionfee']].sample(10))

# 检查 tariff_connectionfee 列的基本统计信息
print("\ntariff_connectionfee 列的基本统计信息:")
print(df['tariff_connectionfee'].describe())


tariff_connectionfee 列的空值数量(填充前):
413078

tariff_connectionfee 列的空值数量(填充后):
0

填充后的数据样本:
       station_name  tariff_amount  tariff_connectionfee
340965        50713           0.15                   1.6
472451        51628           0.45                   0.0
410305        51766           0.30                   0.0
588668        62268           0.40                   0.0
194437        52710           0.40                   0.0
143434        61736           0.40                   0.0
619034        61719           0.45                   0.0
108665        60762           0.45                   0.0
408002        52223           0.15                   1.6
535783        61141           0.55                   0.0

tariff_connectionfee 列的基本统计信息:
count    654118.000000
mean          0.124746
std           0.347083
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           2.000000
Name: tariff_connectionfee, dtype: float64


In [43]:
# 显示原始数据的行数
print(f"原始数据行数: {len(df)}")

# 检查每列中 'UNKNOWN' 的数量
columns_to_check = ['max_chargerate', 'plug_type', 'connector_type']
for col in columns_to_check:
    unknown_count = (df[col] == 'UNKNOWN').sum()
    print(f"{col} 中 'UNKNOWN' 的数量: {unknown_count}")

# 删除包含 'UNKNOWN' 的行
df_cleaned = df[~df[columns_to_check].isin(['UNKNOWN']).any(axis=1)]

# 显示清理后的数据行数
print(f"\n清理后的数据行数: {len(df_cleaned)}")

# 再次检查是否还有 'UNKNOWN' 值
for col in columns_to_check:
    unknown_count = (df_cleaned[col] == 'UNKNOWN').sum()
    print(f"{col} 中剩余 'UNKNOWN' 的数量: {unknown_count}")

# 显示清理后的数据样本
print("\n清理后的数据样本:")
print(df_cleaned[columns_to_check + ['tariff_amount']].sample(10))

# 将 max_chargerate 转换为数值类型（如果还不是的话）
df_cleaned['max_chargerate'] = pd.to_numeric(df_cleaned['max_chargerate'], errors='coerce')

# 显示清理后数据的基本统计信息
print("\n清理后数据的基本统计信息:")
print(df_cleaned[columns_to_check + ['tariff_amount']].describe())

# 更新原始 DataFrame
df = df_cleaned.copy()

原始数据行数: 654118
max_chargerate 中 'UNKNOWN' 的数量: 212
plug_type 中 'UNKNOWN' 的数量: 4240
connector_type 中 'UNKNOWN' 的数量: 212

清理后的数据行数: 649666
max_chargerate 中剩余 'UNKNOWN' 的数量: 0
plug_type 中剩余 'UNKNOWN' 的数量: 0
connector_type 中剩余 'UNKNOWN' 的数量: 0

清理后的数据样本:
       max_chargerate    plug_type connector_type  tariff_amount
107721              7  type_2_plug             AC       0.300000
629257             22  type_2_plug             AC       0.450000
375848             22  type_2_plug             AC       0.400000
177979             22  type_2_plug             AC       0.410033
138356             22  type_2_plug             AC       0.250000
539256              7  type_2_plug             AC       0.450000
351681             22  type_2_plug             AC       0.450000
577647              7  type_2_plug             AC       0.400000
174813              7  type_2_plug             AC       0.300000
520932             22  type_2_plug          Rapid       0.700000

清理后数据的基本统计信息:
       max_chargera

C:\Users\85407\AppData\Local\Temp\ipykernel_19288\2060580324.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['max_chargerate'] = pd.to_numeric(df_cleaned['max_chargerate'], errors='coerce')


In [45]:
# 确保 datetime 列是日期时间格式
df['datetime'] = pd.to_datetime(df['datetime'])

# 按 station_name 和 datetime 排序
df = df.sort_values(['station_name', 'datetime'])

# 检查 weather 列的空值数量（处理前）
print("Weather 列空值数量（处理前）:")
print(df['weather'].isnull().sum())

# 定义一个函数来填充 weather 列的空值
def fill_weather(group):
    group['weather'] = group['weather'].fillna(method='ffill')
    group['weather'] = group['weather'].fillna(method='bfill')
    return group

# 对每个充电站分组应用填充函数
df = df.groupby('station_name').apply(fill_weather)

# 检查 weather 列的空值数量（处理后）
print("\nWeather 列空值数量（处理后）:")
print(df['weather'].isnull().sum())



Weather 列空值数量（处理前）:
304568


C:\Users\85407\AppData\Local\Temp\ipykernel_19288\1555002950.py:13: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  group['weather'] = group['weather'].fillna(method='ffill')
C:\Users\85407\AppData\Local\Temp\ipykernel_19288\1555002950.py:14: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  group['weather'] = group['weather'].fillna(method='bfill')



Weather 列空值数量（处理后）:
0


In [47]:
# 定义一个函数来去除字符串两端的引号
def remove_quotes(text):
    if isinstance(text, str):
        return text.strip("'\"")
    return text

# 应用函数到 weather 列
df['weather'] = df['weather'].apply(remove_quotes)

# 检查处理结果
print("处理后的 weather 列唯一值：")
print(df['weather'].unique())

# 显示天气种类及其频率
print("\n天气种类及其频率：")
print(df['weather'].value_counts())

# 显示包含引号的天气记录（如果还有的话）
quotes = df[df['weather'].str.contains("'|\"", na=False)]
if not quotes.empty:
    print("\n仍然包含引号的记录：")
    print(quotes[['station_name', 'datetime', 'weather']])
else:
    print("\n所有引号已成功移除。")

# 显示处理后的数据样本
print("\n处理后的数据样本：")
print(df[['station_name', 'datetime', 'weather']].sample(10))

处理后的 weather 列唯一值：
['Rain' 'Clouds' 'Fog' 'Clear' 'Drizzle' 'Haze' 'Mist']

天气种类及其频率：
weather
Clouds     570884
Rain        58843
Clear       18363
Fog          1180
Drizzle       286
Haze           89
Mist           21
Name: count, dtype: int64

所有引号已成功移除。

处理后的数据样本：
                    station_name            datetime weather
station_name                                                
50670        425846        50670 2024-06-19 08:00:00  Clouds
61722        620164        61722 2024-06-20 08:00:00  Clouds
51842        472151        51842 2024-06-18 14:00:00  Clouds
50918        353291        50918 2024-06-17 03:00:00  Clouds
53900        154611        53900 2024-06-18 18:00:00  Clouds
54004        2441          54004 2024-06-16 04:00:00  Clouds
52524        293268        52524 2024-06-20 12:00:00  Clouds
60844        85699         60844 2024-06-18 06:00:00  Clouds
60044        12409         60044 2024-06-16 08:00:00    Rain
61775        616419        61775 2024-06-18 20:00:00  Clouds

In [49]:
import os
from datetime import datetime

# 获取当前日期和时间
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")

# 创建输出文件名
output_filename = f"cleaned_charging_station_data_{current_time}.csv"

# 确保输出目录存在
output_dir = "cleaned_data"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 构建完整的输出路径
output_path = os.path.join(output_dir, output_filename)

# 将数据框输出到CSV文件
df.to_csv(output_path, index=False)

print(f"数据已成功输出到文件：{output_path}")

数据已成功输出到文件：cleaned_data\cleaned_charging_station_data_20240710_193353.csv
